In [1]:
# Uniswap math functions
# https://youtu.be/_asFkMz4zhw?t=25 - Uniswap V3 - Liquidity | DeFi

def get_liquidity_0(x, sp, sb):
    return x * sp * sb / (sb - sp)

def get_liquidity_1(y, sp, sa):
    return y / (sp - sa)

def get_liquidity(x, y, sp, sa, sb):
    if sp <= sa:
        liquidity = get_liquidity_0(x, sp, sb)
    elif sp < sb:
        liquidity0 = get_liquidity_0(x, sp, sb)
        liquidity1 = get_liquidity_1(y, sp, sa)
        liquidity = min(liquidity0, liquidity1)
    else:
        liquidity = get_liquidity_1(y, sp, sa)
    return liquidity


#
# Calculate x and y given liquidity and price range
#
def calculate_x(L, sp, sa, sb):
    sp = max(min(sp, sb), sa)     # if the price is outside the range, use the range endpoints instead
    return L * (sb - sp) / (sp * sb)

def calculate_y(L, sp, sa, sb):
    sp = max(min(sp, sb), sa)     # if the price is outside the range, use the range endpoints instead
    return L * (sp - sa)

def calculate_a1(L, sp, sb, x, y):
    # https://www.wolframalpha.com/input/?i=solve+L+%3D+y+%2F+%28sqrt%28P%29+-+a%29+for+a
    # sqrt(a) = sqrt(P) - y / L
    return (sp - y / L) ** 2

def calculate_a2(sp, sb, x, y):
    # https://www.wolframalpha.com/input/?i=solve+++x+sqrt%28P%29+sqrt%28b%29+%2F+%28sqrt%28b%29++-+sqrt%28P%29%29+%3D+y+%2F+%28sqrt%28P%29+-+a%29%2C+for+a
    # sqrt(a) = (y/sqrt(b) + sqrt(P) x - y/sqrt(P))/x
    #    simplify:
    # sqrt(a) = y/(sqrt(b) x) + sqrt(P) - y/(sqrt(P) x)
    sa = y / (sb * x) + sp - y / (sp * x)
    return sa ** 2

#
# Two different ways how to calculate p_b. calculate_b1() uses liquidity as an input, calculate_b2() does not.
#
def calculate_b1(L, sp, sa, x, y):
    # https://www.wolframalpha.com/input/?i=solve+L+%3D+x+sqrt%28P%29+sqrt%28b%29+%2F+%28sqrt%28b%29+-+sqrt%28P%29%29+for+b
    # sqrt(b) = (L sqrt(P)) / (L - sqrt(P) x)
    return ((L * sp) / (L - sp * x)) ** 2

def calculate_b2(sp, sa, x, y):
    # find the square root of b:
    # https://www.wolframalpha.com/input/?i=solve+++x+sqrt%28P%29+b+%2F+%28b++-+sqrt%28P%29%29+%3D+y+%2F+%28sqrt%28P%29+-+sqrt%28a%29%29%2C+for+b
    # sqrt(b) = (sqrt(P) y)/(sqrt(a) sqrt(P) x - P x + y)
    P = sp ** 2
    return (sp * y / ((sa * sp - P) * x + y)) ** 2



def calculate_P(x,y,sa,sb):
  p = sb*x
  q = y-x*sa*sb
  r = -sb*y
  return (math.pow(-1*q+(q**2 - 4*p*r),0.5)/(2*p) )

def tick_price(t):
  return 1.0001**t

def price_tick(p):
  return math.log(p,1.0001)

def getYFromX(x,sa,sb,sp):
    return x * (sp - sa) * sp *sb /(sb - sp)

# ? P is Y/X
def simulator_calc_L(x,pa,pb,p):
    sa = pa ** 0.5
    sb = pb ** 0.5
    sp = p ** 0.5
    if p >= pb:
        y=x
        x=0
        L = get_liquidity(x, y, sp, sa, sb)
    elif p <= pa:
        y = 0
    else:
        print("sim2", sa, sb, sp, x)
        y = getYFromX(x, sa, sb, sp)
        L = get_liquidity(x, y, sp, sa, sb)

    return L,y

# ! Requires X token to be a stable token
def getXY(capitalDollar, token0Decimals, token1Decimals, Pxusd, Pyusd, sp, sa, sb):
    # captal = x * Pxusd / xdecimals + y * Pyusd / yDecimals  ----- eq1
    # replace y with x from getYFromX

    factor1 = Pxusd / (10 ** token0Decimals)
    factor2 = Pyusd / (10 ** token1Decimals)
    factor3 = (sp - sa) * sp * sb / (sb - sp)

    x = capitalDollar / (factor1 + factor3 * factor2)
    y = getYFromX(x, sa, sb, sp)
    return x, y